<a href="https://colab.research.google.com/github/tejlibre/ubiquant-challenge/blob/main/this_is_not_financial_advice_draft_noteboook_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Competition: Ubiquant Market Prediction
## Make predictions against future market data


<img src='https://portal.fgv.br/sites/portal.fgv.br/files/styles/noticia_geral/public/noticias/11/21/mercado-financeiro.jpg?itok=fuZgDZBa'>

Image courtesy of FGV

## Goal:
In this competition sponsored by Ubiquant, a model needs to be built to forecast an investment's return rate, whereby the training and testing of the algorithm will be carried on anonymised historical prices. If successful, an improvement in the ability of quantitative researchers to forecast returns will be derived from the model built. This will enable investors at any scale to make better decisions.


## Context:
Ubiquant Investment (Beijing) Co., Ltd is a leading domestic quantitative hedge fund based in China. Established in 2012, they are committed to creating long-term stable returns for investors. Regardless of any investment strategy, fluctuations are expected in the financial market. Despite this variance, professional investors try to estimate their overall returns. Risks and returns differ based on investment types and other factors, which impact stability and volatility. To attempt to predict returns, there are many computer-based algorithms and models for financial market trading. Yet, with new techniques and approaches, data science could improve quantitative researchers' ability to forecast an investment's return.

## Team members
Nmeso, Antsa, Tejuswi, Akhil

## Evaluation

Submissions are evaluated on the mean of the Pearson correlation coefficient for each time ID.

Pearson correlation coefficient (PCC, also referred to as Pearson's r or bivariate correlation) is a measure of linear correlation between two sets of values. The formulae is denoted as follows:

$\huge r_{xy} = \frac{n\sum{xy} - \sum{x} \sum{y}} {\sqrt{n \sum{x^2} - \left(\sum{x}\right)^2} {\sqrt{n \sum{y^2} -\left(\sum{y}\right)^2}}}$

* $r_{xy}$ = PCC
* $n$ = Number of samples
* $x$ = First set of values
* $y$ = Second set of values

As can be seen from the above equation, PCC is the ratio between the covariance of two variables and the product of their standard deviations. The result always has a value between −1 and 1. As with covariance itself, the measure can only reflect a linear correlation of variables, and ignores many other types of relationship or correlation.

Mean of Pearson correlation coefficient across time IDs can be expressed as follows:

$\huge \textit{Mean} \quad r_{xy} = {\frac{1}{T} \sum_{i=1}^{T}} t_i r_{xy}$

* $T$ = Number of time IDs
* $t_i r_{xy}$ = ith time ID's PCC
[[](http://)](http://)

## Methodology:
<img src='https://raw.githubusercontent.com/akhil-gun/DSI/master/Pipeline_v2.drawio.png'>


### General necessary imports

In [ ]:
# import some necessary packages
import math
import pandas as pd
import numpy as np
from pathlib import Path
import random
import tqdm
from tqdm.notebook import tqdm
import time
import datetime
from datetime import datetime
from scipy.signal import find_peaks

from argparse import Namespace
import random
import os
import gc
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# setting up options
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
from cycler import cycler


## Exploratory Data Analysis

**Folder and files given:**

* `ubiquant/` - The image delivery API that will serve the test set. You may need Python 3.7 and a Linux environment to run the example test set through the API offline without errors.

* `example_sample_submission.csv` - An example submission file provided so the publicly accessible copy of the API provides the correct data shape and format.

* `example_test.csv` - Random data provided to demonstrate what shape and format of data the API will deliver to your notebook when you submit.

* `train.csv` - Train dataset. Further descibe later in the notebook.



In [ ]:
args = Namespace(
    seed=21,
    folds=5,
    workers=4,
    samples=2500000,
    data_path=Path("../input/ubiquant-parquet/"),
)


def reduce_mem_usage(df):
    '''Function to reduce memory usage of dataframe.
    
    Reference: https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline''' 

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

train = reduce_mem_usage(pd.read_parquet(args.data_path.joinpath("train_low_mem.parquet")))
train = pd.read_parquet(args.data_path.joinpath("train_low_mem.parquet"))


FEATURES = [col for col in train.columns if col not in ['target', 'row_id']]
NUM_FEATURES = [col for col in train.columns if col not in ['target', 'row_id', 'investment_id', 'time_id']]
CAT_FEATURES = [feature for feature in FEATURES if feature not in NUM_FEATURES]

inv_ids = random.choices(train['investment_id'].unique(), k=3)

#train = pd.read_csv('/kaggle/input/ubiquant-market-prediction/train.csv', nrows=10000)
# inv_ids = random.choices(train['investment_id'].unique(), k=3)

# train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
# inv_ids = random.choices(train['investment_id'].unique(), k=3)

In [ ]:
# Getting an initial overview of dataset's number of rows, columns, types, and memory usage
train.info()

**Columns of train.csv:**

* ```row_id```  - A unique identifier for the row.

* ```time_id``` - The ID code for the time the data was gathered. The time IDs are in order, but the real time between the time IDs is not constant and will likely be shorter for the final private test set than in the training set.

* ```investment_id``` - The ID code for an investment. Not all investment have data in all time IDs.

* ```target``` - The target (Anonymized).

* ```[f_0:f_299]``` - Anonymized features generated from market data.

In [ ]:
# Checking the first 10 elements of the dataframe, where the colour scale corresponds to a higher value being redder/darker
train.head(10).style.background_gradient(cmap='Reds')

In [ ]:
# Checking the last 10 elements of the dataframe, where the colour scale corresponds to a higher value being redder/darker
train.tail(10).style.background_gradient(cmap='Reds')

In [ ]:
print('Rows in train dataset:', train.shape[0])
print('Columns in train dataset:', train.shape[1])

In [ ]:
print(f"Range of investment ids from {train.investment_id.min()} to {train.investment_id.max()}")

In [ ]:
time_ids  = train.time_id.nunique()
investment_ids = train.investment_id.nunique()
print(f"Number of unique time ids: {time_ids}")
print(f"Number of unique investment ids: {investment_ids}")

In training set, there are 3579 unique investments and private test set will include new unseen investment ids. Furthermore, the investment id range is bigger than the number of investment ids themselves, meaning that some investment ids are repeated but for different time ids.

In [ ]:
# Checking if any of the columns have missing values
print('Missing values in train dataset:', sum(train.isnull().sum()))

In [ ]:
# Counting unique values to see if null values are represented by some other value such as median
train_unique = train.nunique(axis=0)

In [ ]:
# Plotting previous cell results
def plot_unique(nu):
    '''Plotting the number of unique values per row_id in a bar chart'''
    #nu = train_unique.reset_index().sort_values(by = 0)
    nu.columns = ['column name','number of unique entries']

    plt.figure(figsize=(40,15))
    plt.xticks(rotation=90)
    chart = sns.barplot(x='column name', y='number of unique entries', data=nu)

    chart.set_title("Frequency of unique entries per column")
    chart.axhline(train.shape[0], color = "red")
    
nu = train_unique.reset_index().sort_values(by = 0)
plot_unique(nu)

From the bar chart above, it can be seen that about a quarter of the dataset have less than half the amount of unique values compared to the total number of rows [red line]. This suggests that their values might be more categorical that thought of.

In [ ]:
# Saving features per mask for further investigation 
def extracting_nunique(nu, threshold):
    """Extracting the assets depending on threshold value.
    Saving in two separate dataframes. 
    Taking a random of 25 samples of each """
    nu = train_unique.reset_index().sort_values(by = 0)
    nu.columns = ['column_name','number_of_unique_entries']

    #dropping 'time_id', 'investment_id', 'row_id', 'target'
    nu = nu[nu['column_name'].str.split('_').str[0] == 'f']
    nu.info

    mask1 = nu['number_of_unique_entries'] < threshold
    mask2 = nu['number_of_unique_entries'] > threshold

    #print(nu[mask1])
    #nu[mask1].shape[0]

    #print(nu[mask2])
    #nu[mask2].shape[0]

    print('Total number of features: ', nu[mask1].shape[0] + nu[mask2].shape[0])
    #print(nu[mask1].shape[0] + nu[mask2].shape[0] == nu.shape[0])
    
    df_sample1 =  nu[mask1].sample(n=25)
    df_sample2 =  nu[mask2].sample(n=25)

    #df_sample1.column_name.astype('string')
    #df_sample1 = df_sample1.values.tolist()
    df_sample1 = df_sample1['column_name'].tolist()
    df_sample2 = df_sample2['column_name'].tolist()

#     print(df_sample1)
#     print(df_sample2)

    return df_sample1, df_sample2

nu = train_unique.reset_index().sort_values(by = 0)
threshold = 2 * 10**6

df_sample1, df_sample2 = extracting_nunique(nu, threshold)

### Chinese Stock market anlysis

Following this [discussion](https://www.kaggle.com/c/ubiquant-market-prediction/discussion/309720) on Kaggle, it was noticed that the historical dataset given might be during the 2015 stock market crash, began  on the 12th of June 2015 and ended in early February 2016.  [[Wikipedia]](https://en.wikipedia.org/wiki/2015%E2%80%932016_Chinese_stock_market_turbulence) [2]

In [ ]:
# Extracting chinese public holidays from 2014 to 2030.
calendar_df = pd.read_csv("../input/chineseholidays/holidays_of_china_from_2014_to_2030.csv", parse_dates=["date"], 
                          date_parser=lambda x: datetime.strptime(x, '%Y-%m-%d'))
#display(calendar_df.head())

# leave only national holidays
calendar_df = calendar_df.loc[(calendar_df.type.isin(["National holiday", "Common local holiday"]))]
#display(calendar_df.head())

# fill with everyday from 2014 to 2022
calendar_df = (
    pd.DataFrame({"date": pd.date_range(start="2014-01-01", end="2022-01-01")}).merge(calendar_df, on="date", how="left")
    .assign(weekday=lambda x: x.date.dt.day_name(), year=lambda x: x.date.dt.year)
)
#display(calendar_df.head())

# remove weekends and national holidays and align with time_id
calendar_df = (
    calendar_df.loc[(~calendar_df.weekday.isin(["Sunday", "Saturday"]))&(calendar_df.name.isna())]
    .reset_index(drop=True)
    .head(train.time_id.max()+1)
    .dropna(axis=1)
)
#display(calendar_df.head())

In [ ]:
f, (ax0, ax1, ax2) = plt.subplots(3, 1, gridspec_kw={'height_ratios': [1,1,6]}, figsize=(10,10), sharex=True, dpi=128)
_df = (
    train[['time_id', 'investment_id']]
    .groupby("time_id")
    .count()
    .reindex(range(train.time_id.min(), train.time_id.max()+1))
    .set_index(calendar_df.date)
)
peeks, _ = find_peaks(-_df.values.squeeze(), threshold=200)
_df.plot(ax=ax0)
ax0.set_xticks(ticks=peeks, minor=True)
ax0.set_ylabel("count")
ax0.legend(loc='upper left')

(
    train[['time_id', 'target']]
    .groupby("time_id")
    .mean()
    .reindex(range(train.time_id.min(), train.time_id.max()+1))
    .set_index(calendar_df.date)
    .plot(ax=ax1)
)
ax1.axvspan(*mdates.date2num(_df.loc[(_df.index>"2015-06")&(_df.index<"2016-03")].index[[0,-1]]), fill=True, alpha=0.9, color="#eee")
ax1.set_ylabel("mean")
ax1.legend(loc='upper left')

_df = (
    train[['investment_id', 'time_id', "target"]]
    .pivot_table(index="time_id", columns="investment_id", values="target", aggfunc="count")
    .reindex(range(train.time_id.min(), train.time_id.max()+1))
    .set_index(calendar_df.date)
)
ax2.imshow(_df.T, cmap='winter', interpolation='nearest', aspect="auto", origin="lower", alpha=0.6, 
           extent=[*mdates.date2num([calendar_df.date.min(), calendar_df.date.max()]),train.investment_id.min(), 
                   train.investment_id.max()])
ax2.set_xlabel("date")
ax2.set_ylabel("investment_id")
ax2.xaxis_date()
plt.tight_layout()
plt.show()

## Features exploration

There are 300 anonymized continuous features in dataset and they are named from f_0 to f_299. 

All of the features are zero-centered and they have standard deviation of one since they are standardized during the anonymisation process. Most of the features have symmetrical normal distributions but some of them have very extreme outliers which are skewing their distributions.

Feature means and standard deviations vary between different time IDs and investments. It looks like feature means and standard deviations are dependent to time. They make sharp transitions on some periods. Feature standard deviations are more likely to make sharp transitions on different periods however feature mean outliers are observed in the same period most of the time. Feature means and standard deviations per investment looks randomly distributed among investments because it is related to those investment's time IDs.

In [ ]:
# Defining function to visualise set of 25 features in 5 x 5 grid

# features = list(train.columns[4:29])    #   0:24
# # features = list(train.columns[29:54])   #  25:49
# # features = list(train.columns[54:79])   #  50:74
# # features = list(train.columns[79:104])  #  75:99
# # features = list(train.columns[104:129]) # 100:124
# # features = list(train.columns[129:154]) # 125:149
# # features = list(train.columns[154:179]) # 150:174
# # features = list(train.columns[179:204]) # 175:199
# # features = list(train.columns[204:229]) # 200:224
# # features = list(train.columns[229:254]) # 225:249
# # features = list(train.columns[254:279]) # 250:274
# # features = list(train.columns[279:304]) # 275:300

def visualize_feature(features):
    plt.rcParams['figure.dpi'] = 600
    fig = plt.figure(figsize=(10, 10), facecolor='#f6f5f5')
    gs = fig.add_gridspec(5, 5)
    gs.update(wspace=0.3, hspace=0.3)
    background_color = '#f6f5f5'
    run_no = 0

    colormap = ['#1DBA94','#1C5ED2', '#FFC300', '#C70039']
    plt.rc('axes', prop_cycle=(cycler('color', colormap)))

    for row in range(0, 5):
        for col in range(0, 5):
            locals()["ax"+str(run_no)] = fig.add_subplot(gs[row, col])
            locals()["ax"+str(run_no)].set_facecolor(background_color)
            for s in ["top","right"]:
                locals()["ax"+str(run_no)].spines[s].set_visible(False)
            run_no += 1  


    # Kernel density plots to visualise distribution of each features
    run_no = 0
    for col in features:
        sns.kdeplot(ax=locals()["ax"+str(run_no)], x=train[col], zorder=2, alpha=1, linewidth=1, color='#ffd514')
        sns.kdeplot(ax=locals()["ax"+str(run_no)], x=train[train['investment_id'].isin(inv_ids)][col], 
                    hue=train[train['investment_id'].isin(inv_ids)]['investment_id'],zorder=2, alpha=1, fill=True, 
                    color=colormap, linewidth=0.5, legend=False,palette=colormap[:3], hue_order=inv_ids.sort(reverse=True))

        locals()["ax"+str(run_no)].grid(which='major', axis='x', zorder=0, color='#EEEEEE', linewidth=0.4)
        locals()["ax"+str(run_no)].grid(which='major', axis='y', zorder=0, color='#EEEEEE', linewidth=0.4)
        locals()["ax"+str(run_no)].set_ylabel('')
        locals()["ax"+str(run_no)].set_xlabel(col, fontsize=4, fontweight='bold')
        locals()["ax"+str(run_no)].tick_params(labelsize=4, width=0.5)
        locals()["ax"+str(run_no)].xaxis.offsetText.set_fontsize(4)
        locals()["ax"+str(run_no)].yaxis.offsetText.set_fontsize(4)
        #locals()["ax"+str(run_no)].get_legend().remove()

        run_no += 1

    plt.show()
    
    return


# Choose series of 25 features to graph 
features = list(df_sample1)
visualize_feature(features)

In [ ]:
# Choose series of 25 features to graph 
features = list(df_sample2)
visualize_feature(features)

In [ ]:
del train, nu, df_sample1, df_sample2, features

# Removing outliers

## using quantile
if you have analysis residual of you model, you can saw that he badly approximate outside ~ 5 and 95 quantile.
In addition some work clarified, that it is problem related to disbalance investmend_ids on time_ids.
If you run a regular catboost baseline, you will probably run into this problem.
Let's see what features depend on outside 5 and 95 quantiles.

references: https://www.kaggle.com/lucamassaron/eda-target-analysis

In [ ]:
train = pd.read_csv('/kaggle/input/ubiquant-market-prediction/train.csv', nrows=30000)

In [ ]:
print(train['target'].count())
threshold_right = train['target'].quantile(0.95)
threshold_left = train['target'].quantile(0.05)

print(threshold_right)
print(threshold_left)

train = train[(train['target'] > threshold_left) & (train['target'] < threshold_right)]
print(train['target'].count())


In [ ]:
df_x = train.drop(['target'], axis =1)
df_y = train['target'] 

# Split the dataset

X_test_model, y_test_model is the testing set to be used at the end of model training for evaluating model predictions

X_train, X_test, y_train, y_test : this is the training set and validation set for feature selection.
It will be recombined and used as training set for the model



In [ ]:
X_train_model, X_test_model, y_train_model, y_test_model = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split( X_train_model, y_train_model, test_size=0.2, random_state=42)


# Time series split

### Acknowledgements

https://www.kaggle.com/ionanicleoid/intro-to-comparing-ml-models-lgbm-tuning

https://www.kaggle.com/robikscube/ubiquant-parquet

https://www.kaggle.com/melanie7744/understanding-the-submission-api-for-newbies

Testing a first random forest model
To be used later for feature importance

### Import data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

import sklearn
from sklearn.model_selection import TimeSeriesSplit
import time

print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
%%time
df = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
example_test = pd.read_csv('/kaggle/input/ubiquant-market-prediction/example_test.csv')
example_sample_submission =  pd.read_csv('/kaggle/input/ubiquant-market-prediction/example_sample_submission.csv')

In [ ]:
features = [f'f_{i}' for i in range(300)]

In [ ]:
DROP_BEFORE = 600

df = df[df["time_id"] > DROP_BEFORE].reset_index(drop=True)
df.shape

In [ ]:
X = np.array(df.drop(['target'], axis = 1))
y = np.array(df['target'])

In [ ]:
from sklearn.model_selection import TimeSeriesSplit


def time_series_split(n,train_size):
    """
    Time Series cross-validator
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals, in train/test sets.
    In each split, test indices must be higher than before, and
    thus shuffling in cross validator is inappropriate.

    This cross-validation object is a variation of KFold.
    In the kth split, it returns first k folds as train set and
    the (k+1)th fold as test set.

    Parameters
    ----------
    n, default=5
        Number of splits. Must be at least 2.
    max_train_size(int), default=None
        Maximum size for a single training set.
    Returns
    ----------
    
    References
    ----------
    .. [1] https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html
    """
    
    ts_cv = TimeSeriesSplit(
        n_splits=5,
        max_train_size=train_size,
        test_size=200000,
        gap=200000
    )
    
    return ts_cv

ts_cv = time_series_split(5,1000000)

In [ ]:
x=pd.DataFrame(X)

In [ ]:
all_splits = list(ts_cv.split(X, y))

In [ ]:
feature_names = [f"feature {i}" for i in range(X.shape[1])]

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

for fold in all_splits:
    xtrain, ytrain = x.iloc[fold[0]], df.iloc[fold[0]]['target']
    xval, yval = x.iloc[fold[1]], df.iloc[fold[1]]['target']
    x_train.append(xtrain)
    y_train.append(ytrain)
    x_test.append(xval)
    y_test.append(yval)

In [ ]:
import pickle as pkl
#to save it
with open("train_time_split.pkl", "wb") as f:
    pkl.dump([x_train, y_train, x_test, y_test], f)

In [ ]:
#to load it
#with open("train_time_split.pkl", "rb") as f:
#    x_train, y_train, x_test, y_test = pkl.load(f)

## Applying scaling

### Acknowledgements
1. https://www.kaggle.com/kartushovdanil/ubiquant-market-prediction-eda#4.-Features
2. https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline
3. https://www.kaggle.com/ilialar/ubiquant-eda-and-baseline

In [ ]:
## Separate Normally and Skewed Distributed Features
normal_features = [1 , 2 , 6 , 9 , 20 , 21, 24
                  ,28 , 35 , 36 , 40 , 43 ,
                  50 , 51 , 57 , 67 , 69 , 72,
                  75 , 76 ,82 ,85 , 86 ,
                  90 , 93 , 94 , 96 , 98 ,
                  103 , 105 , 109 , 106 , 114 ,116,
                  125 , 126 , 130 , 133 , 134 ,135 ,139,140 , 141 ,144,146,
                  141 ,171,
                  178 , 180 , 185 , 189 , 192 , 194 ,195 ,199,
                  205 , 206 ,212 ,213 , 217 , 221 ,222,223,
                  226 , 230 , 239 ,242 ,
                   252 , 254 ,256 , 259 ,261 , 266 ,273,
                  276 , 283 , 285 , 290 , 297]


normal_features_list = ['f_'+str(i) for i in normal_features]


print("Number of Normally dist features approx are",len(normal_features_list))

all_features_list = ['f_'+str(i) for i in range(0 , 300)]
skewed_features_list = list(set(all_features_list).difference(set(normal_features_list)))
print("Number of Skewed dist features approx are",len(skewed_features_list))
## Check for mistakes
print(set(skewed_features_list).intersection(set(normal_features_list)))

In [ ]:
scaler_normal = StandardScaler()
scaler_outlier = RobustScaler()

X_train[normal_features_list] = scaler_normal.fit_transform(X_train[normal_features_list])
X_train[skewed_features_list] = scaler_outlier.fit_transform(X_train[skewed_features_list])

In [ ]:
X_train.head()

In [ ]:
## Always apply scaler transform on validation data separately
X_test[normal_features_list] = scaler_normal.transform(X_test[normal_features_list])
X_test[skewed_features_list] = scaler_outlier.transform(X_test[skewed_features_list])

# Order reduction using PCA

From our litterature review, we found that combining PCA with ANN gives better accuracy in the prediction. In this notebook, we are implementing PCA to make a feature selection for our data.

PCA gives the best possible representation of a dataset while reducing the dimension. This transformation our data in such a way that the first principal component has the largest possible variance (that is, accounts for as much of the variability in the data as possible), and each succeeding component, in turn, has the highest possible variance possible.

Some of the codes were taken from [this notebook](https://www.kaggle.com/ollibolli/lgb-pca-train-5-fold) and this [other one](https://www.kaggle.com/miguelangelnieto/pca-and-regression). We also read [this blogpost](https://www.reneshbedre.com/blog/principal-component-analysis.html).

The processing of the data follows from [here](https://www.kaggle.com/nmesoegwuekwe/preprocessings)

Goals :

Feature reduction with PCA
Data transformation  
Test different regression models to illustrate the difference.

### Making an instance of the PCA class and fit it to our data.

### First, we will work without the time_id as feature and see what happens.

In [ ]:
pca_out = PCA().fit(X_train)

### Proportion of variance


In this part, we are viewing how much each principal component contributes to the variance of our dataset.

In [ ]:
pca_out.explained_variance_ratio_ 

### Cummulative proportion of variance

Now, let us cumulate the proportion to see which combination of the PCs contribute mostly in the variance. That will helps us to be able the PCs with low information and hence, reduce the dimension.

In [ ]:
cumsum = np.cumsum(pca_out.explained_variance_ratio_)
cumsum

In [ ]:
# component loadings or weights (correlation coefficient between original variables and the component) 
# component loadings represents the elements of the eigenvector
# the squared loadings within the PCs always sums to 1
loadings = pca_out.components_
num_pc = pca_out.n_features_
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
loadings_df['variable'] = X_train.columns.values
loadings_df = loadings_df.set_index('variable')
loadings_df.head()

### PCs retention
We will only keep the PCS that explain the most variance, in our case, we will take only 90 % (it is very subjective). 

In [ ]:
def pca_to_retain(threshold) :
    '''This function take as input a threshold t and return the number of PCs contributes to the proportion t of the variance'''
    for i in range(len(cumsum)) :
        if cumsum[i]>threshold :
            break
    return i

### Transforming the datasets

Now that we decide which combination of PCs to keep, we need to transform our training dataset. If we want to use the PCA in a model, we need also to apply the same transformation to the test dataset.

In [ ]:
def transform(dataset, pca_model = pca_out, num_pca = 100) :
    '''This function transforms the dataset using the trained PCA model'''
    return pca_model.transform(dataset)[:,:num_pca ]

In [ ]:
X_train_PCA = transform(X_train) 

X_test_PCA = transform(X_test)

### Viewing the new PCs dataset.

Let us see how the transformed dataset looks like.

In [ ]:
pd.DataFrame(X_train_PCA).head()

**Here we can import the transformed dataset in a new file**



In [ ]:
del loadings_df # to free-up memory

* ### Working with the original data, without PCA.

# LGBM for feature selection

LGBM stands for Light Gradient Boosting Machine. LGBM is a fast and efficient gradient boosting framework based on decision tree algorithm. LGBM is a similar boosting method to XGBoost but is especially different in how it creates the tree or base learners. 

Unlike other ensemble techniques, LGBM grows trees leaf-wise, which can reduce loss during the sequential boosting process. This usually results in higher accuracy than other boosting algorithms. 

### Import some necessary packages

In [ ]:
import math
import pandas as pd
import numpy as np
from pathlib import Path
import random
import tqdm
from tqdm.notebook import tqdm
import time
import datetime
from datetime import datetime
from scipy.signal import find_peaks

from argparse import Namespace
import random
import os
import gc
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import pickle

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
print("scikit-learn version: {}". format(sklearn.__version__))

import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm import early_stopping, log_evaluation, record_evaluation
print("LightGBM version:  {}".format(lgb.__version__))

# setting up options
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
from cycler import cycler

from scipy.stats import pearsonr

!python --version

### Loading the dataset

In [ ]:
df_train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')

display(df_train.shape)
display(df_train.info())
df_train.head()
#df_train.tail()

### Basic dataset preprocessing

Filtering the dataset, whereby assets after time_id of 600 are used (after the gap in the data found in the EDA). 

In [ ]:
first_time_id_to_use = 600 
features_to_use= [col for col in df_train.columns if col.startswith("f")] # use only the anonymised features
time_id_to_split_train_and_val = 1000


df_train = df_train.loc[df_train.time_id >= first_time_id_to_use]
print("df_train.shape: ",df_train.shape)

X_train = df_train.loc[df_train.time_id < time_id_to_split_train_and_val]
X_val = df_train.loc[df_train.time_id >= time_id_to_split_train_and_val]
y_train = X_train.target
y_val = X_val.target
X_train = X_train[features_to_use]
X_val = X_val[features_to_use]
print("X_train.shape:  ", X_train.shape)
print("X_val.shape:    ", X_val.shape)

In [ ]:
del df_train #free up memory
gc.collect()

### Defining the LGBM model

In [ ]:
def LBGM_model(X_train, y_train, X_val, y_val):
    """Implement Scikit-learn's LGBM model
    Also returns useful evaluation metrics such as MSE, RMSE, Pearson's coefficient, 
    and execution time of the model.
    
    Read more in the :ref:`Docs <https://lightgbm.readthedocs.io/en/latest/Python-Intro.html#training>`.
   
    Parameters
    ----------
    X_train : pandas dataframe
        Features subset of dataset used for training.
    y_train : pandas dataframe
        Target of training subset.
    X_val : pandas series
        Features subset of dataset used for validation/testing.
    y_val : pandas series  
        Target of validation/testing subset.

    Returns
    -------
    model:
    
    metric_over_time: Dictionary
        Dictionary of evalution of metrics.
    mse: float
        Mean-squared error.
    rmse: float
        Root-mean squared error.
    pcc: float
        Pearson correlation coefficient.
    execution_time: float
        Time for model to execute [s].
    References
    ----------
       [1] `Microsoft Corporation, (2000). LightGBM Python-package introduction
       <https://lightgbm.readthedocs.io/en/latest/Python-Intro.html#training>`_.
       [2] `Scikit-learn, (2021). sklearn.metrics.mean_squared_error
       <https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html>`_.
    """
    # Create LGBM datasets
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)
    
    # Parameters not finetuned yet
    lgb_params = {'objective': 'regression',
        'metric': 'MSE',
        'boosting_type': 'gbdt',
        'lambda_l1': 2.3e-05,
        'lambda_l2': 0.1,
        'num_leaves': 4,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.9,
        'bagging_freq': 7,
        'min_child_samples': 20,
        'num_iterations': 1000
                 }
    
    ts = time.time() # Initialising time for measuring execution time

    metric_over_time = {} # Dict for logging the evaluation metrics

    model = lgb.train(        
            lgb_params, 
            dtrain, 
            valid_sets=[dtrain, dval],
            valid_names=['Train','Validation'],
            callbacks=[early_stopping(100), log_evaluation(100), record_evaluation(metric_over_time)]
        )

    execution_time = time.time() - ts
    
    print("\nTraining time of model: " + str(round(execution_time, 3)) + "s")
    
    y_val_hat = model.predict(X_val)

    mse = mean_squared_error(y_val, y_val_hat, squared=True)
    rmse = mean_squared_error(y_val, y_val_hat, squared=False)

    print('\nEvalution on val data:')
    # Using MSE as a proxy for pearson correlation (https://www.kaggle.com/c/ubiquant-market-prediction/discussion/302181)
    print("MSE:  ", round(mse, 5))
    print("RMSE: ", round(rmse, 5))
    
    # Getting Pearson's correlation coeff. 
    pcc, _ = pearsonr(y_val_hat, y_val)
    #print(pearsonr(y_val_hat, y_val))
    print("PCC on validation data: ", round(pcc, 5))
    
    # Plotting metric over time
    lgb.plot_metric(metric_over_time, figsize=(20,10))
    plt.show()
    
    return model, metric_over_time, mse, rmse, pcc, execution_time  
    

### Applying the LGBM model 

In [ ]:
model1, metric_over_time1, mse1, rmse1, pcc1, execution_time1 = LBGM_model(X_train, y_train, X_val, y_val)

In the graph above, we can see how the metric improves as the training goes on. The mean squared error (MSE), also referred to as square loss or l2, is decreasing steadily.

LightGBM has a nice build in function for plotting the feature importance. Feature importance can be displayed as "gain", showing the total gains of splits which use the feature, or "split", showing the numbers of times the feature is used in a model.

In [ ]:
# let's look at which features lgbm deems important
# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.plot_importance.html
lgb.plot_importance(model1, figsize=(20,40), importance_type='gain', max_num_features=300) # importance_type: gain/split: V7 has 'split'
plt.show()

In [ ]:
# Getting rid of features with little importance
imp = pd.DataFrame({'Value':model1.feature_importance(importance_type='gain'),'Feature':X_train.columns}).sort_values(by="Value",ascending=False).reset_index(drop=True)

#imp.Value.value_counts()
imp = imp[imp.Value>100]  # remove all features with gain lower than 100
new_feature_list = list(imp.Feature)
print("Number of features, new: ", len(new_feature_list))

In [ ]:
# Save model1 to disk
filename = 'lgbm_model_pre_feat_selection.sav'
pickle.dump(model1, open(filename, 'wb'))

In [ ]:
del model1, imp # free up memory

### Re-applying LGBM after feature selection

In [ ]:
model2, metric_over_time2, mse2, rmse2, pcc2, execution_time2 = LBGM_model(X_train[new_feature_list], y_train, X_val[new_feature_list], y_val)

### Checking the change in evaluation metrics between both models, pre- vs post- feature selection

In [ ]:
def get_percentage_change(current_param, previous_param):
    """Return the percentage change of a parameter with respect to initial parameter value
    
    Parameters
    ----------
    current_param : float/int
        Current parameter to compare.
    previosu_param, : float/int
        Previous version of parameter to compare.

    Returns
    -------
    %_change : float/int
        The percentage change of both parameters w.r.t. previous state.
    """
    if current_param == previous_param:
        return 0
    try:
        return round(((current_param - previous_param) / previous_param) * 100.0, 5)
    except ZeroDivisionError:
        return float('inf')

    

In [ ]:
print("Change in MSE: {} %".format(get_percentage_change(mse2, mse1)))
print("Change in RMSE: {} %".format(get_percentage_change(rmse2, rmse1)))
print("Change in PCC: {} %".format(get_percentage_change(pcc2, pcc1)))
print("Change in execution time: {} %".format(get_percentage_change(execution_time2, execution_time1)))

It can be seen that there is a small decrease in the MSE and RMSE, but an increase in the PCC, when the model is only applied using the features selected. So while MSE, RSME and PCC are comparable between LightGBM  using all 300 features and LightGBM using only the more important features, the training time is greatly reduced! 

In [ ]:
# Save model2 to disk
filename = 'lgbm_model_post_feat_selection.sav'
pickle.dump(model2, open(filename, 'wb'))

In [ ]:
del model2

# Random forest for feature selection

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import shap
from matplotlib import pyplot as plt

from pathlib import Path
import time

import random
import os
import gc
import seaborn as sns
from matplotlib import pyplot as plt



#from sklearn.linear_model import LinearRegression
#print("scikit-learn version: {}". format(sklearn.__version__))

In [ ]:
# Import competition data

train = pd.read_csv('../input/ubiquant-market-prediction/train.csv', nrows=10000)
example_test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
example_sample_submission =  pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')

## Scaling
https://scikit-learn.org/stable/modules/preprocessing.html

Note : 6.3.1.3. Scaling data with outliers If your data contains many outliers, scaling using the mean and variance of the data is likely to not work very well. In these cases, you can use RobustScaler as a drop-in replacement instead. It uses more robust estimates for the center and range of your data.

In [ ]:
df = train
scaler = StandardScaler()
X = np.array(df.drop(['row_id', 'time_id', 'investment_id', 'target'], axis = 1))
scaler.fit(X)
X = scaler.transform(X)

y = np.array(df['target'])

In [ ]:
def reduce_mem_usage(df):
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
     
    return df

df = reduce_mem_usage(df)

In [ ]:
features = [f'f_{i}' for i in range(300)]
#print(features)
df_median = df[features].head(30000).median()
#print(df_median)

## RandomForestRegressor


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Timeseries split from sklearn - work in progress

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(
    n_splits=5,
    #gap=48,
    max_train_size=10000,
    #test_size=1000,
)
x=pd.DataFrame(X)
all_splits = list(ts_cv.split(X, y))

View the data in splits

In [ ]:
train_3, test_3 = all_splits[3]
#x.iloc[test_3]

In [ ]:
train_4, test_4 = all_splits[4]
#x.iloc[test_4]

## Mean decrease in impurity - gini importance
https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

Feature importances are provided by the fitted attribute featureimportances and they are computed as the mean and standard deviation of accumulation of the impurity decrease within each tree.

In [ ]:
%%time
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_confidence = rf.score(X_test, y_test)
rf_confidence

In [ ]:
start_time = time.time()
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

In [ ]:
feature_names = [f"feature {i}" for i in range(X.shape[1])]

In [ ]:
forest_importances = pd.Series(importances, index=features)
#forest_importances.sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(20,100))
forest_importances.sort_values(ascending=True).plot(kind="barh", fontsize=12)
plt.ylabel('MDI')
plt.show()

## Permutation importances
Permutation feature importance overcomes limitations of the impurity-based feature importance: they do not have a bias toward high-cardinality features and can be computed on a left-out test set.


In [ ]:
from sklearn.inspection import permutation_importance
#imp = permutation_importance(rf, X_train, y_train, oob_regression_r2_score)


start_time = time.time()
result = permutation_importance(
    rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

forest_importances = pd.Series(result.importances_mean, index=features)

In [ ]:
plt.figure(figsize=(20,100))
forest_importances.sort_values(ascending=True).plot(kind="barh", fontsize=12)
plt.ylabel('permutation')
plt.show()

## XGBoost for feature selection

### Importing the librairies

In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import warnings
import xgboost as xgb
import pickle
#import lightgbm as lgb
from sklearn.model_selection import train_test_split

### Loading the dataset.

In [ ]:
%%time
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
train.head()

### Taking the list of features 

In [ ]:
L = [ x for x in train.columns if x != 'target' ]

### splitting the dataset into test and train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split((train.head(300000))[L], (train.head(300000))['target'], test_size=0.2, random_state=42)

### Creating DMatrix 

We need to put our dataset in the format XGBoost works with.

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

### Setting the parametters of the model

In [ ]:
params = {
 'booster': 'gbtree',
    'max_depth': 5, 
    'learning_rate': 0.1,
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'objective': 'binary:hinge',
    'rate_drop': 0.1,
    'n_estimators': 500,
    'silent' :1
}

num_rounds = 20

### Watchlist

Watchlist is a list of xgb.DMatrix, each of them is tagged with name.
Watchlist allows us to monitor the evaluation result on all data in the list

In [ ]:
watchlist = [(dtest, 'test'), (dtrain,'train')]

### Training the model.

In [ ]:
bst = xgb.train(params, dtrain, num_rounds, watchlist)

### Dumping the model to a txt file to use later.

In [ ]:
# to see how the model looks
#bst.dump_model('dump.raw.txt')

### Plotting the Fscore of each feature

In [ ]:
xgb.plot_importance(bst)

### List of feature

In [ ]:
L = bst.get_score(importance_type='gain')

### Sort the list of feature in decreasing order.

In [ ]:
sortedfeatures = dict(sorted(L.items(), key=lambda item: item[1]),reverse=True)

The sorted features in order of importance. Showing first 5 of the dictionary.

In [ ]:
list(sortedfeatures.items())[:5]

# RAPIDS Random Forest
Using cudf and cuml librairies 

In [ ]:
import cudf

df = cudf.read_parquet("../input/ubiquant-parquet/train_low_mem.parquet")
print(df.shape)
df.head()

In [ ]:
df["time_id"].max(), df.shape

In [ ]:
DROP_BEFORE = 600

df = df[df["time_id"] > DROP_BEFORE].reset_index(drop=True)
df.shape

In [ ]:
DROP_AFTER = 1000

df = df[df["time_id"] < DROP_AFTER].reset_index(drop=True)
df.shape

In [ ]:
import cupy
import cuml

print("cuML version:", cuml.__version__)

WINDOW = 20
START = 700
N_SPLITS = 15

cv = []

for i in range(N_SPLITS):
    train_ind = cupy.where(df["time_id"].values <= START + i*WINDOW)[0]
    val_ind = cupy.where((df["time_id"].values > START + i*WINDOW) & (df["time_id"].values <= START + (i+1)*WINDOW))[0]
    cv.append((cupy.asnumpy(train_ind), cupy.asnumpy(val_ind)))
    print(len(train_ind), len(val_ind))

In [ ]:
features = [col for col in df.columns if col not in {"row_id", "target", "investment_id", "time_id"}]
features += ["investment_te"]
len(features)

In [ ]:
class RAPIDSModel:
    def __init__(self):
        self.te = cuml.preprocessing.TargetEncoder()
        self.rf = cuml.ensemble.RandomForestRegressor(n_estimators=256, split_criterion="mse", bootstrap=True,
                                                      max_samples=0.6, min_samples_leaf=64, max_features=0.6, n_bins=512)
        
    def calculate_sample_weight(self, train_df):
        time_mean = train_df.groupby("time_id")["target"].mean().reset_index().rename(columns={"target": "target_mean"})
        time_std = train_df.groupby("time_id")["target"].std().reset_index().rename(columns={"target": "target_std"})

        train_df = train_df.merge(time_mean, on="time_id", how="left").merge(time_std, on="time_id", how="left")
        train_df = train_df.sort_values(["time_id", "investment_id"]).reset_index(drop=True)

        train_df["norm_target"] = (train_df["target"] - train_df["target_mean"])/train_df["target_std"]
        train_df["sw"] = (train_df["norm_target"].abs() + 1)/2
                
        return train_df
        
    def fit(self, train_df):
        train_df["investment_te"] = self.te.fit_transform(train_df["investment_id"], train_df["target"]).astype("float32")
        #train_df = self.calculate_sample_weight(train_df)
        
        #self.svr.fit(train_df[features], train_df["target"], sample_weight=train_df["sw"])
        self.rf.fit(train_df[features], train_df["target"])

        return self
        
    def predict(self, test_df):
        test_df["investment_te"] = self.te.transform(test_df["investment_id"]).astype("float32").get()
        #return 0.7*self.rf.predict(test_df[features]) + 0.3*self.svr.predict(test_df[features])
        return self.rf.predict(test_df[features])

In [ ]:
from tqdm import tqdm


def evaluate(val_df):
    scores = []
    for time_id in val_df["time_id"].unique().values_host:
        time_df = val_df[val_df["time_id"] == time_id]
        scores.append(time_df["target"].corr(time_df["pred"]))

    return cupy.mean(cupy.array(scores))


val_scores = []


for f, (train_ind, val_ind) in tqdm(enumerate(cv), total=len(cv)):
    train_df, val_df = df.iloc[train_ind], df.iloc[val_ind]

    model = RAPIDSModel().fit(train_df)
    y_pred = model.predict(val_df)
    val_df["pred"] = y_pred.values
    
    val_scores.append(evaluate(val_df).item())
    
val_scores = cupy.array(val_scores)

In [ ]:
print("Validation scores:", val_scores)
print("Mean:", cupy.mean(val_scores))
print("STD:", cupy.std(val_scores))

In [ ]:
model = RAPIDSModel().fit(df)

### Pickling random forest model
https://docs.rapids.ai/api/cuml/nightly/pickling_cuml_models.html

In [ ]:
pickle.dump(model, open("ubiquant_cuml_rf_model.pkl", "wb"))

In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
%%time
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
feature_columns = ['investment_id', 'time_id'] + features
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
train.head()

In [ ]:
investment_id = train.pop("investment_id")
investment_id.head()

In [ ]:
_ = train.pop("time_id")

In [ ]:
y = train.pop("target")
y.head()

## Create a IntegerLookup layer for investment_id input

In [ ]:
%%time
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
with tf.device("cpu"):
    investment_id_lookup_layer.adapt(investment_id)

## Make Tensorflow dataset

In [ ]:
import tensorflow as tf
def preprocess(X, y):
    print(X)
    print(y)
    return X, y
def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(256)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

## Modeling

Creaing dnn models to be used for the ensemble modelling. The swish activation function as it outperforms ReLu with diverse size of batches


In [ ]:
def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model
def get_model2():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)    
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
   # investment_id_x = layers.Dropout(0.65)(investment_id_x)
   
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.65)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
   # x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
  #  x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.75)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model
def get_model3():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    #investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1)(x)
    output = tf.keras.layers.BatchNormalization(axis=1)(output)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

In [ ]:
#del train
#del investment_id
del y
gc.collect()

Adding the 3 model gotten to the list 'models' for running an inference on the trained model and finally makig the prediction on the hidden test set using the kaggle api given

In [ ]:
models = []
for i in range(5):
    model = get_model()
    model.load_weights(f'../input/dnn-base/model_{i}')
    models.append(model)
for i in range(10):
    model = get_model2()
    model.load_weights(f'../input/train-dnn-v2-10fold/model_{i}')
    models.append(model)
    
    
for i in range(10):
    model = get_model3()
    model.load_weights(f'../input/dnnmodelnew/model_{i}')
    models.append(model)

In [ ]:
def preprocess_test(investment_id, feature):
    return (investment_id, feature), 0
def make_test_dataset(feature, investment_id, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds
def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
for (test_df, sample_prediction_df) in iter_test:
    ds = make_test_dataset(test_df[features], test_df["investment_id"])
    sample_prediction_df['target'] = inference(models, ds)
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)

# Conclusion

This was a complex project to tackle from the beginninng, given the large dataset and its accompanying memory issues. The EDA revealed some interesting observations such as about a quarter of the data (mostly of the features) has less than half the amount of unique values compared to the other features suggesting that some features might be more categorical than initially thought of. Moreover, it was found that some gaps in the data could be correlated to the 2015 Chinese bubble burst. 

Four different order reduction methods were explored, namely PCA, LGBM, Random Forest, and XGBoost. 

Ensemble model is incomplete, we planned to have an ensemble based on models from different approaches, including gradient boosting, decision trees, and deep neural networks.

The best kaggle score achieved was of 0.153 

# Recommendations for future works
Since the main constraint was time, several options could not be explored. In the future, the following options can be considered for further potential improvements in the results:
* Better selection of features
* Better preprocessing
* More attention to the time series nature of the dataset
* Hyperparameter finetuning
* Better ensemble model. Adding weights to the contribution of each model? Possibility of finetuning of weights

## References 

[1] TORCH ME. 'The most advanced analytics' (2022). Kaggle notebook URL: https://www.kaggle.com/kartushovdanil/the-most-advanced-analytics

[2] VALLEY. 'Ubiquant LGBM Baseline' (2022).Kaggle notebook URL: https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline

[3] GUNES EVITAN. 'Ubiquant Market Prediction - EDA' (2022). Kaggle notebook URL: https://www.kaggle.com/gunesevitan/ubiquant-market-prediction-eda